In [2]:
from tira.rest_api_client import Client
from tqdm import tqdm
import gzip
import json

tira = Client()

In [3]:
def du(path):
    import subprocess
    return subprocess.check_output(['du','-s', path]).split()[0].decode('utf-8')

def ds_stats(path):
    queries = 0
    docs = 0
    with gzip.open(path / 'corpus.jsonl.gz', 'rt') as f:
        for l in f:
            docs += 1
    with open(path / 'queries.jsonl', 'r') as f:
        for l in f:
            queries += 1
    return {"docs_count": docs, "queries_count": queries}


In [4]:
EMBEDDINGS = ["naver-splade-v3", "webis-splade", "naver-splade-v3-distilbert", "naver-splade_v2_distil", 
              "naver-splade-v3-doc", "castorini-unicoil-noexp-msmarco-passage", "opensearch-project-opensearch-neural-sparse-encoding-doc-v2-mini",
              "opensearch-project-opensearch-neural-sparse-encoding-doc-v3-distill", "naver-splade-v3-lexical",
              "opensearch-project-opensearch-neural-sparse-encoding-doc-v2-distill", "opensearch-project-opensearch-neural-sparse-encoding-v2-distill"]

DATASETS = ["trec-robust-2004-fold-5-20250926-test", "trec-robust-2004-fold-4-20250926-test", "trec-robust-2004-fold-3-20250926-test", 
            "trec-robust-2004-fold-2-20250926-test", "trec-robust-2004-fold-1-20250927-test", "trec-33-rag-20250926_1-training", 
            "trec-29-deep-learning-passages-20250926-training", "trec-28-misinfo-20251008_1-test", 
            "trec-28-deep-learning-passages-20250926-training", "trec-23-web-20251008-test",
            "trec-22-web-20251008-test", "trec-21-web-20251008-test", "trec-20-web-20251008-test",
            "trec-19-web-20251008-test", "trec-18-web-20251008-test", 
            "tiny-example-20251002_0-training"
             ]

SIZES = {}

for dataset in DATASETS:
    embedding_sizes = {}
    for embedding in tqdm(EMBEDDINGS, dataset):
        embedding_sizes[embedding] = du(tira.get_run_output(f'lsr-benchmark/lightning-ir/{embedding}', dataset))
    SIZES[dataset] = {
        'dataset-size': du(tira.download_dataset(task='lsr-benchmark', dataset=dataset)),
        'dataset_stats': ds_stats(tira.download_dataset(task='lsr-benchmark', dataset=dataset)),
        "embedding-sizes": embedding_sizes}

with open('../lsr_benchmark/datasets/overview.json', 'w') as f:
    f.write(json.dumps(SIZES))

tiny-example-20251002_0-training: 100%|██████████| 11/11 [00:01<00:00,  5.51it/s]
